In [1]:
#!/usr/bin/env python3
import pandas as pd
import lz4.frame
import gzip
import io
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from glob import glob
from plumbum.cmd import rm
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

#from keras import backend as K
#print(K.tensorflow_backend._get_available_gpus())

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def plotline(data):
    plt.figure()
    plt.plot(data)
    plt.legend()
    plt.show()

def event_count(time_series, data_name):
    time_series = time_series[['Fill Price (USD)']].values
    upevents = 0
    downevents = 0
    sameprice = 0
    prev_obv = time_series[0]
    for obv in time_series[1:]:
        if obv > prev_obv:
            upevents += 1
        elif obv < prev_obv:
            downevents += 1
        elif obv == prev_obv:
            sameprice += 1
        prev_obv = obv
    print('=== Event counts on %s ===' % data_name)
    print('upevents')
    print(upevents)
    print('downevents')
    print(downevents)
    print('sameprice')
    print(sameprice)
    print()

def mse(time_series, data_name):
    time_series = time_series[['Fill Price (USD)']].values
    total_squared_error = 0
    total_absolute_error = 0
    prev_obv = time_series[0]
    for obv in time_series[1:]:
        total_squared_error += (obv - prev_obv)**2
        total_absolute_error += abs(obv - prev_obv)
        prev_obv = obv
    num_predictions = len(time_series) - 1
    mean_squared_error = total_squared_error / num_predictions
    mean_absolute_error = total_absolute_error / num_predictions
    root_mean_squared_error = np.sqrt(mean_squared_error)
    print('=== baseline on %s ===' % data_name)
    print('total squared error')
    print(total_squared_error)
    print('total absolute error')
    print(total_absolute_error)
    print('mean squared error')
    print(mean_squared_error)
    print('mean absolute error')
    print(mean_absolute_error) 
    print('root mean squared error')
    print(root_mean_squared_error) 
    print()

In [3]:
def show_summary_statistics():
    #event_count(small_set, 'small')
    train_set = df.iloc[0:num_samples_training]
    dev_set = df.iloc[num_samples_training:num_samples_training+num_samples_dev]
    test_set = df.iloc[num_samples_training+num_samples_dev:]
    event_count(train_set, 'train')
    event_count(dev_set, 'dev')
    event_count(test_set, 'test')
    mse(train_set, 'train')
    mse(dev_set, 'dev')
    mse(test_set, 'test')
#show_summary_statistics()

In [4]:
def preprocess(data):
    values = np.array(data)
    values = values.reshape(-1,1)
    values = values.astype('float32') 
    return values

In [5]:
def plot_losses(model_history, title):
    plt.figure()
    plt.plot(model_history.history['loss'], label='Train')
    plt.plot(model_history.history['val_loss'], label='Dev')
    plt.xlabel('Epochs'); plt.ylabel('Loss (mse)')
    plt.title(title)
    plt.legend(); plt.show()

In [6]:
def inverse_transform_pricescaler(data, Y_prevrawprice, fitted_scaler):
    return fitted_scaler.inverse_transform(preprocess(data))

def inverse_transform_percentdiff(data, Y_prevrawprice, fitted_scaler=None):
    orig_prices = Y_prevrawprice
    change = orig_prices * data
    return orig_prices + change
    #return fitted_scaler.inverse_transform(preprocess(data))

#print(Y_test_prevrawprice)
#print(inverse_transform_percentdiff(Y_test, Y_test_prevrawprice))

inverse_transform = inverse_transform_percentdiff

In [7]:
def plot_predictions(model, X_test, Y_test, Y_prevrawprice, title, inverse=False, scaler=None):
    y_hat = model.predict(X_test)

    if inverse:
        y_hat = inverse_transform(y_hat, Y_prevrawprice, scaler)
        Y_test = inverse_transform(Y_test, Y_prevrawprice, scaler)

    plt.plot(y_hat, label='Predicted')
    plt.plot(Y_test, label='True')
    plt.xlabel('Time'); 

    if inverse:
        plt.ylabel('Price')
    else:
        plt.ylabel('RESCALED Price')

    plt.title(title)
    plt.legend(); plt.show()

In [8]:
def calculate_MSE_RMSE(model, scaler, X_test, Y_test, Y_prevrawprice, model_name):
    y_hat = model.predict(X_test)
    y_hat_inverse = inverse_transform(y_hat, Y_prevrawprice, scaler)
    Y_test_inverse = inverse_transform(Y_test, Y_prevrawprice, scaler)
    mse = mean_squared_error(Y_test_inverse, y_hat_inverse)
    rmse = np.sqrt(mean_squared_error(Y_test_inverse, y_hat_inverse))
    print('%s:' % model_name)
    print('Test MSE: %.3f' % mse)
    print('Test RMSE: %.3f' % rmse)
    print()

In [9]:
def train_evaluate(model, model_name, 
                   X_train, Y_train, Y_train_prevrawprice, X_dev, Y_dev, Y_dev_prevrawprice, X_test, Y_test, Y_test_prevrawprice,
                   lag=10, batch_size=100, epochs=10, verbose=1):

    # Train model
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      validation_split=0.05, verbose=verbose, shuffle=False)
    #train_evaluate_showresults(history, model, model_name, 
    #                 X_train, Y_train, X_dev, Y_dev, X_test, Y_test,
    #                 lag, batch_size, epochs, verbose)
    return history

In [10]:
def train_evaluate_showresults(history, model, model_name, 
                   X_train, Y_train, Y_train_prevrawprice, X_dev, Y_dev, Y_dev_prevrawprice, X_test, Y_test, Y_test_prevrawprice,
                   lag=10, batch_size=100, epochs=10, verbose=1):
    # Plot losses, predictions, and calculate MSE and RMSE
    plot_losses(history, 'Loss\n(%s)' % model_name)
    plot_predictions(model, X_dev, Y_dev, Y_dev_prevrawprice, 'Test Predictions\n(%s)' % model_name)
    plot_predictions(model, X_dev, Y_dev, Y_dev_prevrawprice, 'Test Predictions\n(%s)' % model_name, inverse=True, scaler=price_scaler)
    calculate_MSE_RMSE(model, price_scaler, X_dev, Y_dev, Y_dev_prevrawprice, '%s' % model_name)

In [11]:
def evaluate_test(model, model_name, 
                   X_train, Y_train, Y_train_prevrawprice, X_dev, Y_dev, Y_dev_prevrawprice, X_test, Y_test, Y_test_prevrawprice,
                   lag=10, batch_size=100, epochs=10, verbose=1):
    # Plot losses, predictions, and calculate MSE and RMSE
    #plot_losses(history, 'Loss\n(%s)' % model_name)
    plot_predictions(model, X_test, Y_test, Y_test_prevrawprice, 'Test Predictions\n(%s)' % model_name)
    plot_predictions(model, X_test, Y_test, Y_test_prevrawprice, 'Test Predictions\n(%s)' % model_name, inverse=True, scaler=price_scaler)
    calculate_MSE_RMSE(model, price_scaler, X_test, Y_test, Y_test_prevrawprice, '%s' % model_name)

In [12]:
'''
def initialize_model(X_train, loss, optimizer, num_LSTMs, num_units, dropout):
    
    LSTM_input_shape = [X_train.shape[1], X_train.shape[2]]

    # DEFINE MODEL
    model = Sequential()

    if num_LSTMs == 2:
            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))

            model.add(LSTM(num_units[1], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))
        
    if num_LSTMs == 3:
            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))

            model.add(LSTM(num_units[1], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))
            
            model.add(LSTM(num_units[2], return_sequences=False))

    #model.add(Dense(1))
    model.add(TimeDistributed(Dense(1), input_shape=(LSTM_input_shape)))
    #model.add(Dense(1, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
    model.add(Activation('linear'))

    
    model.compile(loss=loss, optimizer=optimizer)
    
    return model
'''

"\ndef initialize_model(X_train, loss, optimizer, num_LSTMs, num_units, dropout):\n    \n    LSTM_input_shape = [X_train.shape[1], X_train.shape[2]]\n\n    # DEFINE MODEL\n    model = Sequential()\n\n    if num_LSTMs == 2:\n            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))\n            model.add(Dropout(dropout))\n\n            model.add(LSTM(num_units[1], input_shape=LSTM_input_shape, return_sequences=True))\n            model.add(Dropout(dropout))\n        \n    if num_LSTMs == 3:\n            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))\n            model.add(Dropout(dropout))\n\n            model.add(LSTM(num_units[1], input_shape=LSTM_input_shape, return_sequences=True))\n            model.add(Dropout(dropout))\n            \n            model.add(LSTM(num_units[2], return_sequences=False))\n\n    #model.add(Dense(1))\n    model.add(TimeDistributed(Dense(1), input_shape=(LSTM_input_shape)))\n    #mo

In [13]:
#geza
def initialize_model(X_train, loss, optimizer, num_LSTMs, num_units, dropout):
    
    #LSTM_input_shape = [X_train.shape[1], X_train.shape[2]]
    LSTM_input_shape = [X_train.shape[1], X_train.shape[2]]
    print('input shape is')
    print(LSTM_input_shape)

    # DEFINE MODEL
    model = Sequential()

    if num_LSTMs == 2:
            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))

            model.add(LSTM(num_units[1], return_sequences=True))
        
    if num_LSTMs == 3:
            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))

            model.add(LSTM(num_units[1], return_sequences=True))
            model.add(Dropout(dropout))
            
            model.add(LSTM(num_units[2], return_sequences=True))

    #model.add(Dense(1))
    model.add(TimeDistributed(Dense(1)))
    #model.add(Dense(1, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
    model.add(Activation('linear'))

    
    model.compile(loss=loss, optimizer=optimizer)
    
    return model

In [14]:
'''
def load_data():
    files = sorted(glob('cboe/parquet_preprocessed_BTCUSD/*.parquet'))
    all_dataframes = []
    for file in files:
        print(file)
        df = pq.read_table(file).to_pandas()
        all_dataframes.append(df)
    result = pd.concat(all_dataframes)
    pq.write_table(pa.Table.from_pandas(result), 'cboe/parquet_preprocessed_subset_only_BTCUSD.parquet', compression='snappy')
    df = pq.read_table('cboe/parquet_preprocessed_subset_only_BTCUSD.parquet').to_pandas();
    print(df.dtypes)
    print(df.shape)
    return df
'''

"\ndef load_data():\n    files = sorted(glob('cboe/parquet_preprocessed_BTCUSD/*.parquet'))\n    all_dataframes = []\n    for file in files:\n        print(file)\n        df = pq.read_table(file).to_pandas()\n        all_dataframes.append(df)\n    result = pd.concat(all_dataframes)\n    pq.write_table(pa.Table.from_pandas(result), 'cboe/parquet_preprocessed_subset_only_BTCUSD.parquet', compression='snappy')\n    df = pq.read_table('cboe/parquet_preprocessed_subset_only_BTCUSD.parquet').to_pandas();\n    print(df.dtypes)\n    print(df.shape)\n    return df\n"

In [15]:
#geza
import os.path

def load_data():
    if not os.path.isfile('cboe/parquet_preprocessed_subset_only_BTCUSD_merged.parquet'):
        files = sorted(glob('cboe/parquet_preprocessed_BTCUSD_merged/*.parquet'))
        all_dataframes = []
        for file in files:
            print(file)
            df = pq.read_table(file).to_pandas()
            all_dataframes.append(df)
        result = pd.concat(all_dataframes)
        pq.write_table(pa.Table.from_pandas(result), 'cboe/parquet_preprocessed_subset_only_BTCUSD_merged.parquet', compression='snappy')
    df = pq.read_table('cboe/parquet_preprocessed_subset_only_BTCUSD_merged.parquet').to_pandas();
    print(df.dtypes)
    print(df.shape)
    return df

In [16]:
def split_X(df):
    n_all = df.shape[0]
    n_train = round(n_all * 0.9)
    n_dev   = round(n_all * 0.05)
    n_test  = round(n_all * 0.05)
    print('n_all:  ', n_all)
    print('n_train:', n_train)
    print('n_dev:  ', n_dev)
    print('n_test: ', n_test)
    
#     X_train = df.iloc[:n_train, 1:-1].values.astype('float32')[:, None, :]
#     X_dev   = df.iloc[n_train:n_train+n_dev, 1:-1].values.astype('float32')[:, None, :]
#     X_test  = df.iloc[n_train+n_dev:, 1:-1].values.astype('float32')[:, None, :]
#     X_train = df.iloc[:n_train, 1:-1].values.astype('float32')
#     X_dev   = df.iloc[n_train:n_train+n_dev, 1:-1].values.astype('float32')
#     X_test  = df.iloc[n_train+n_dev:, 1:-1].values.astype('float32')
    X_train = df.iloc[:n_train, 1:16].values.astype('float32')
    X_dev   = df.iloc[n_train:n_train+n_dev, 1:16].values.astype('float32')
    X_test  = df.iloc[n_train+n_dev:, 1:16].values.astype('float32')
    print(X_train.shape)
    print(X_dev.shape)
    print(X_test.shape)

    return X_train, X_dev, X_test

In [17]:
def split_Y(df):
    n_all = df.shape[0]
    n_train = round(n_all * 0.9)
    n_dev   = round(n_all * 0.05)
    n_test  = round(n_all * 0.05)
    Y_train = df.iloc[:n_train, -1:].values.astype('float32')
    Y_dev   = df.iloc[n_train:n_train+n_dev, -1:].values.astype('float32')
    Y_test  = df.iloc[n_train+n_dev:, -1:].values.astype('float32')
    print(Y_train.shape)
    print(Y_dev.shape)
    print(Y_test.shape)
    
    return Y_train, Y_dev, Y_test

In [18]:
def df_to_parquet(df, outfile):
    pq.write_table(pa.Table.from_pandas(df), outfile, compression='snappy')

In [66]:
def evaluate_model(model, history, X_train, X_dev, X_test, Y_train, Y_dev, Y_test):
    train_loss = history.history['loss'][-1]
    dev_loss = history.history['val_loss'][-1]
    test_loss = model.evaluate(X_test, Y_test, verbose=0)
    
    y_hat_train = model.predict(X_train)
    y_hat_dev   = model.predict(X_dev)
    y_hat_test  = model.predict(X_test)
    
    train_prop_correct = np.sum(np.sign(y_hat_train) == np.sign(Y_train)) / (Y_train_final.shape[0] * Y_train_final.shape[1])
    dev_prop_correct   = np.sum(np.sign(y_hat_dev)   == np.sign(Y_dev))   / (Y_dev_final.shape[0]   * Y_dev_final.shape[1])
    test_prop_correct  = np.sum(np.sign(y_hat_test)  == np.sign(Y_test))  / (Y_test_final.shape[0]  * Y_test_final.shape[1])
    
    evaluation = [train_loss, dev_loss, test_loss, train_prop_correct, dev_prop_correct, test_prop_correct]
    return evaluation

In [20]:
# df = result
# n_all = df.shape[0]
# n_train = round(n_all * 0.9)
# n_dev   = round(n_all * 0.05)
# n_test  = round(n_all * 0.05)
# print('n_all:  ', n_all)
# print('n_train:', n_train)
# print('n_dev:  ', n_dev)
# print('n_test: ', n_test)

In [21]:
def create_sequenced_data(data, window, step, y=True):
    sequenced = []
    for minute in range(0, len(data) - window, step):
        chunk = data[minute:minute+window]
        sequenced.append(chunk)
    sequenced = np.array(sequenced)
#     if y:
#         sequenced = np.reshape(sequenced, [len(sequenced), window])  
    return sequenced

# def create_sequenced_data_Y(Y, window, step):
#     sequenced = []
#     for minute in range(0, len(data) - window, step):
#         chunk = data[minute:minute+window]
#         sequenced.append(chunk)
#     Y_train_final = np.reshape(Y_train_final, [len(Y_train_final), 60]).shape
#     return np.array(sequenced)



# Y_train = df.iloc[:len(df), -1:].values.astype('float32')
# Y_train_final = create_sequenced_data(Y_train, 60, 1)
# Y_train_final = np.reshape(Y_train_final, [len(Y_train_final), 60]).shape

In [41]:
files = sorted(glob('cboe/parquet_preprocessed_BTCUSD_merged/*.parquet'))
all_dataframes = []
for file in files:
    df = pq.read_table(file).to_pandas()
    all_dataframes.append(df)
df = pd.concat(all_dataframes)

In [34]:
'''
X_train, X_dev, X_test = split_X(df)
Y_train, Y_dev, Y_test = split_Y(df)

X_train_final = create_sequenced_data(X_train, window=60, step=1, y=False)
X_dev_final   = create_sequenced_data(X_dev, window=60, step=1, y=False)
X_test_final  = create_sequenced_data(X_test, window=60, step=1, y=False)

Y_train_final = create_sequenced_data(Y_train, window=60, step=1, y=True)
Y_dev_final   = create_sequenced_data(Y_dev, window=60, step=1, y=True)
Y_test_final  = create_sequenced_data(Y_test, window=60, step=1, y=True)

print('Train, dev, test shapes:')
print(X_train_final.shape)
print(X_dev_final.shape)
print(X_test_final.shape)
print(Y_train_final.shape)
print(Y_dev_final.shape)
print(Y_test_final.shape)
'''

"\nX_train, X_dev, X_test = split_X(df)\nY_train, Y_dev, Y_test = split_Y(df)\n\nX_train_final = create_sequenced_data(X_train, window=60, step=1, y=False)\nX_dev_final   = create_sequenced_data(X_dev, window=60, step=1, y=False)\nX_test_final  = create_sequenced_data(X_test, window=60, step=1, y=False)\n\nY_train_final = create_sequenced_data(Y_train, window=60, step=1, y=True)\nY_dev_final   = create_sequenced_data(Y_dev, window=60, step=1, y=True)\nY_test_final  = create_sequenced_data(Y_test, window=60, step=1, y=True)\n\nprint('Train, dev, test shapes:')\nprint(X_train_final.shape)\nprint(X_dev_final.shape)\nprint(X_test_final.shape)\nprint(Y_train_final.shape)\nprint(Y_dev_final.shape)\nprint(Y_test_final.shape)\n"

In [44]:
#geza
X_train, X_dev, X_test = split_X(df)
Y_train, Y_dev, Y_test = split_Y(df)

window_size = 60

X_train_final = create_sequenced_data(X_train, window=window_size, step=1, y=False)
X_dev_final   = create_sequenced_data(X_dev, window=window_size, step=1, y=False)
X_test_final  = create_sequenced_data(X_test, window=window_size, step=1, y=False)

Y_train_final = create_sequenced_data(Y_train, window=window_size, step=1, y=True)
Y_dev_final   = create_sequenced_data(Y_dev, window=window_size, step=1, y=True)
Y_test_final  = create_sequenced_data(Y_test, window=window_size, step=1, y=True)

print('Train, dev, test shapes:')
print(X_train_final.shape)
print(X_dev_final.shape)
print(X_test_final.shape)
print(Y_train_final.shape)
print(Y_dev_final.shape)
print(Y_test_final.shape)

print('Train, dev, test shapes original:')
print(X_train.shape)

n_all:   1222293
n_train: 1100064
n_dev:   61115
n_test:  61115
(1100064, 15)
(61115, 15)
(61114, 15)
(1100064, 1)
(61115, 1)
(61114, 1)
Train, dev, test shapes:
(1100044, 20, 15)
(61095, 20, 15)
(61094, 20, 15)
(1100044, 20, 1)
(61095, 20, 1)
(61094, 20, 1)
Train, dev, test shapes original:
(1100064, 15)


In [51]:
batch_size = 8192 #16384 #32768 #4096
epochs = 200
verbose = 2
loss = 'mean_squared_error'
optimizer = 'adagrad' #'adam'
#num_LSTM = 2
#n_units = [256, 256]
num_LSTM = 3
n_units = [256, 256, 256]
dropout = 0.1

model = initialize_model(X_train_final, loss, optimizer, num_LSTM, n_units, dropout)

history = model.fit(X_train_final, Y_train_final, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev_final, Y_dev_final), verbose=verbose, shuffle=False) 

input shape is
[20, 15]
Train on 1100044 samples, validate on 61095 samples
Epoch 1/200
 - 72s - loss: 0.3484 - val_loss: 0.0028
Epoch 2/200
 - 67s - loss: 0.0022 - val_loss: 0.0012
Epoch 3/200
 - 67s - loss: 0.0011 - val_loss: 6.8551e-04
Epoch 4/200
 - 67s - loss: 7.6866e-04 - val_loss: 4.6540e-04
Epoch 5/200
 - 67s - loss: 5.8591e-04 - val_loss: 3.4716e-04
Epoch 6/200
 - 67s - loss: 4.7981e-04 - val_loss: 2.7539e-04
Epoch 7/200
 - 67s - loss: 4.1179e-04 - val_loss: 2.2944e-04
Epoch 8/200
 - 67s - loss: 3.6424e-04 - val_loss: 1.9655e-04
Epoch 9/200
 - 67s - loss: 3.2931e-04 - val_loss: 1.7374e-04
Epoch 10/200
 - 67s - loss: 3.0264e-04 - val_loss: 1.5644e-04
Epoch 11/200


KeyboardInterrupt: 

In [53]:
a = evaluate_model(model, history, X_train_final, X_dev_final, X_test_final, Y_train_final, Y_dev_final, Y_test_final)

In [62]:
Y_test_final.shape[0]*Y_test_final.shape[1]

1221880

In [65]:
history.history['val_loss']

[0.02718626707792282,
 0.006338200066238642,
 0.0027326869312673807,
 0.001698706648312509,
 0.001254145405255258,
 0.0010413419222459197,
 0.0009030214278027415,
 0.0008182916790246964,
 0.0007581285899505019,
 0.0007028873660601676,
 0.0006665494875051081,
 0.0006137048476375639,
 0.0005802287487313151,
 0.0005413988837972283,
 0.0005202949396334589,
 0.0004865903756581247,
 0.00047809717943891883,
 0.00044235767563804984,
 0.00042427933658473194,
 0.00040302457637153566,
 0.0003817896649707109,
 0.00037104933289811015,
 0.00035242902231402695,
 0.00032933190232142806,
 0.0003196826728526503,
 0.00030853855423629284,
 0.0002918703539762646,
 0.0002794977044686675,
 0.00026941325631923974,
 0.0002592851815279573,
 0.00024878818658180535,
 0.00024208147078752518,
 0.00022939304471947253,
 0.0002216892025899142,
 0.00021344481501728296,
 0.0002067625755444169,
 0.0001979375520022586,
 0.00019705621525645256,
 0.00018816158990375698,
 0.0001789871312212199,
 0.0001746904745232314,
 0.000

In [ ]:
batch_size = 8192 #16384 #32768 #4096
epochs = 200
verbose = 2
loss = 'mean_squared_error'
optimizer = 'adam'
#num_LSTM = 2
#n_units = [256, 256]
num_LSTM = 3
n_units = [256, 256, 256]
dropout = 0.1

model = initialize_model(X_train_final, loss, optimizer, num_LSTM, n_units, dropout)

history = model.fit(X_train_final, Y_train_final, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev_final, Y_dev_final), verbose=verbose, shuffle=False) 

In [ ]:
num_epochs = 30
loss = 'mse'
optimizers = ['adagrad', 'adam', 'rmsprop']
batch_sizes = [2048, 8192, 16384]
include_monthDayHour = [True, False]
num_LSTMs = [2,3]
num_units_2 = [[128, 256], [256, 256]]
num_units_3 = [[128, 256, 256], [256, 256, 256], [256, 512, 512]]
#amount_of_data = {'one_year', 'all'}
#dropout = np.random.uniform(0.5, 0.05, num_LSTMs)
dropout = 0.1

# Load data
df = load_data()
X_train, X_dev, X_test = split_X(df) 
Y_train, Y_dev, Y_test = split_Y(df)

df_noTime = df.iloc[:,1:16]
X_train_noTime, X_dev_noTime, X_test_noTime = split_X(df_noTime) 

# Initialize output dataframe
outfile = 'cboe/grid_search_dataSubset.parquet'
columns = ['num_epochs', 'loss', 'optimizer', 'batch_size', 'include_time', 'num_LSTMs', 'num_units',
           'train_loss', 'dev_loss', 'test_loss', 'train_prop_correct', 'dev_prop_correct', 'test_prop_correct']
df_output = pd.DataFrame(columns=columns)
pq.write_table(pa.Table.from_pandas(df_output), outfile, compression='snappy')

In [ ]:
count = 0
for include_time in include_monthDayHour:
    if not include_time:
        X_train = X_train_noTime
        X_dev =   X_dev_noTime
        X_test =  X_test_noTime

    for optimizer in optimizers:
        for batch_size in batch_sizes:
            for num_LSTM in num_LSTMs:
                if num_LSTM == 2:
                    num_units = num_units_2
                elif num_LSTM == 3:
                    num_units = num_units_3
                for n_units in num_units:
                    # Load output dataframe
                    df_output = pq.read_table(outfile).to_pandas()
                    
                    # Initialize model
                    model = initialize_model(X_train, loss, optimizer, num_LSTM, n_units, dropout)
                    
                    # Train model
                    if count%10==0:
                        verbose=1
                        print(count, '/', 120)
                    else:
                        verbose=0
                    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, Y_dev), verbose=verbose, shuffle=False) 
                    
                    # Evaluate model
                    evaluate = evaluate_model(model, history, X_train, X_dev, X_test, Y_train, Y_dev, Y_test)
                    
                    # Write to dataframe and save
                    row = [num_epochs, loss, optimizer, batch_size, include_time, num_LSTM, str(n_units)]
                    row.extend(evaluate)
                    df_output.loc[len(df_output)] = row
                    df_to_parquet(df_output, outfile)
                    
                    count += 1

In [ ]:
# 2048 batch, 3 cells, numunits[2]
# HYPERPARAMETERS
#####################
loss = 'mean_squared_error'
optimizer = 'adagrad'
batch_size = 2048
epochs = 10

num_LSTMs = 3
num_units = num_units_3[2]
model = initialize_model(X_train, loss, optimizer, num_LSTMs, num_units, dropout)
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, Y_dev), verbose=1, shuffle=False)